In [71]:
#Applied Data Science: Capstone
#Coursework 2
#Clustering in Toronto

In [76]:
#Prepare possibly-will-use packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

Solving environment: done

# All requested packages already installed.



In [77]:
#Input the dataset
#Note that the dataset on this wiki link is slightly different from it is given by Coursera
!wget -q -O 'toronto_data.json' https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.

url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'

#Read the table
TData = pd.read_html(url)
TData = TData[0]

In [78]:
#Select the rows that need deleting
Keep = TData[TData['Borough']!='Not assigned']

#Group the data for repeated postcodes
#Keep columns PostCode and Borough, and merge Neighbourhood cells with the same postcodes or boroughs
T = Keep.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
T.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [79]:
#Show the shape of the target dataframe
T.shape

(103, 3)

In [81]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /Users/CharlesYan/opt/anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [89]:
#Read the longitude and latitude table
longlat = pd.read_csv('http://cocl.us/Geospatial_data')
#Change the column name to 'Postcode' as is shown in T
longlat = longlat.rename({'Postal Code': 'Postcode'}, axis=1)

In [96]:
#Merge the dataframe longlat and the dataframe T
TD = T.merge(longlat)
print('The shape the dataframe after merging: ', TD.shape)
TD.head(12)

The shape the dataframe after merging:  (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
